In [ ]:
# Import du système (simplifié pour demo)
import re
from typing import Tuple, List, Dict, Any, Optional
from dataclasses import dataclass

class CyclingSpellChecker:
    """Correcteur orthographique spécialisé cyclisme - Demo Version"""
    
    def __init__(self):
        # Corrections observées en production Vekta
        self.corrections = {
            # Familier → Technique (comme production)
            'fond': 'VO2max', 'max': 'VO2max', 'a fond': 'VO2max',
            'seuille': 'seuil', 'chaude': 'echauffement', 'chauffe': 'echauffement',
            'cool down': 'retour au calme', 'set': 'series', 'sets': 'series',
            'repos': 'recuperation', 'pause': 'recuperation',
            
            # Fautes courantes
            'doie': 'dois', 'minut': 'minutes', 'mn': 'minutes',
            'avk': 'avec', 'apres': 'après'
        }
    
    def correct_text(self, text: str) -> Tuple[str, List[str]]:
        """Corrige le texte avec retour des corrections appliquées"""
        corrected = text.lower()
        corrections_applied = []
        
        for wrong, correct in self.corrections.items():
            if wrong in corrected:
                corrected = corrected.replace(wrong, correct)
                corrections_applied.append(f"'{wrong}' → '{correct}'")
        
        return corrected, corrections_applied

# Test avec phrase familière typique
spell_checker = CyclingSpellChecker()

test_query = "je doie faire 10 minut de chaude, apres 3 set de 5 mn a fond avk 2 mn repos"
print(f"🎯 REQUÊTE ORIGINALE:")
print(f"   '{test_query}'")
print()

corrected_text, corrections = spell_checker.correct_text(test_query)
print(f"✅ APRÈS CORRECTION ({len(corrections)} corrections):")
print(f"   '{corrected_text}'")
print()
print(f"🔧 CORRECTIONS APPLIQUÉES:")
for correction in corrections:
    print(f"   • {correction}")


In [ ]:
@dataclass
class WorkoutSegment:
    """Segment d'entraînement paramétrique"""
    name: str
    duration_minutes: int
    intensity_percent: int  # %FTP
    zone: str
    description: str
    segment_type: str  # 'warmup', 'work', 'recovery', 'cooldown'

@dataclass
class ParametricWorkout:
    """Séance d'entraînement paramétrique"""
    name: str
    description: str
    total_duration: int
    segments: List[WorkoutSegment]
    difficulty: int  # 1-5
    workout_type: str

class ParameterExtractor:
    """Extracteur de paramètres pour génération paramétrique - Version Demo"""
    
    def __init__(self):
        # Zones d'intensité simplifiées
        self.intensity_zones = {
            'recuperation': (40, 55, 'Zone 1'),
            'endurance': (56, 75, 'Zone 2'), 
            'aerobic': (56, 75, 'Zone 2'),
            'tempo': (76, 90, 'Zone 3'),
            'seuil': (91, 105, 'Zone 4'),
            'vo2max': (106, 120, 'Zone 5'),
            'max': (106, 120, 'Zone 5'),
            'neuromuscular': (150, 300, 'Zone 6')
        }
        
        # Valeurs par défaut (approche coach expert)
        self.defaults = {
            'warmup_duration': 10,
            'cooldown_duration': 10,
            'recovery_intensity': 50,
            'work_intensity': 85,
            'default_zone': 'Zone 3'
        }
    
    def extract_workout_parameters(self, corrected_query: str) -> Dict[str, Any]:
        """Extrait les paramètres de la requête corrigée"""
        query = corrected_query.lower()
        
        # Extraction des durées
        durations = self._extract_durations(query)
        
        # Extraction des intensités/zones
        intensities = self._extract_intensities(query)
        
        # Extraction de la structure (séries, répétitions)
        structure = self._extract_structure(query)
        
        # Classification du type de séance
        workout_type = self._classify_workout_type(query, structure)
        
        return {
            'durations': durations,
            'intensities': intensities,
            'structure': structure,
            'workout_type': workout_type,
            'has_warmup': self._has_warmup(query),
            'has_cooldown': self._has_cooldown(query)
        }
    
    def _extract_durations(self, query: str) -> Dict[str, List[int]]:
        """Extrait toutes les durées mentionnées"""
        durations = {
            'warmup': [],
            'work': [],
            'recovery': [],
            'cooldown': [],
            'total': []
        }
        
        # Échauffement
        warmup_match = re.search(r'(\d+)\s*(?:min|minute)s?\s*(?:de\s*)?(?:echauffement|chaude|chauffe|warm)', query)
        if warmup_match:
            durations['warmup'].append(int(warmup_match.group(1)))
        
        # Retour au calme
        cooldown_match = re.search(r'(\d+)\s*(?:min|minute)s?\s*(?:de\s*)?(?:retour|cool|calme)', query)
        if cooldown_match:
            durations['cooldown'].append(int(cooldown_match.group(1)))
        
        # Travail et récupération (séries)
        series_match = re.search(r'(\d+)\s*(?:fois|set|serie).*?(\d+)\s*(?:min|minute)', query)
        if series_match:
            reps = int(series_match.group(1))
            work_duration = int(series_match.group(2))
            durations['work'].extend([work_duration] * reps)
            
            # Récupération entre séries
            recovery_match = re.search(r'(\d+)\s*(?:min|minute)s?\s*(?:de\s*)?(?:repos|recuperation|pause)', query)
            if recovery_match:
                recovery_duration = int(recovery_match.group(1))
                durations['recovery'].extend([recovery_duration] * (reps - 1))
        
        return durations
    
    def _extract_intensities(self, query: str) -> Dict[str, str]:
        """Extrait les intensités/zones mentionnées"""
        intensities = {}
        
        for zone_name, (min_power, max_power, zone_label) in self.intensity_zones.items():
            if zone_name in query:
                intensities[zone_name] = {
                    'power_range': (min_power, max_power),
                    'zone': zone_label,
                    'target_power': (min_power + max_power) // 2
                }
        
        # Détection de pourcentages explicites
        percent_match = re.search(r'(\d+)\s*%', query)
        if percent_match:
            percent = int(percent_match.group(1))
            intensities['explicit_percent'] = {
                'power_range': (percent, percent),
                'zone': self._percent_to_zone(percent),
                'target_power': percent
            }
        
        return intensities
    
    def _extract_structure(self, query: str) -> Dict[str, Any]:
        """Extrait la structure de la séance (séries, pyramides, etc.)"""
        structure = {
            'type': 'continuous',  # ou 'intervals', 'pyramid', 'ladder'
            'intervals': [],
            'pattern': None
        }
        
        # Détection d'intervalles
        intervals_match = re.search(r'(\d+)\s*(?:fois|set|serie|x)', query)
        if intervals_match:
            structure['type'] = 'intervals'
            structure['intervals'] = [int(intervals_match.group(1))]
        
        # Détection de pyramides
        if 'pyramide' in query or 'pyramid' in query:
            structure['type'] = 'pyramid'
        
        return structure
    
    def _classify_workout_type(self, query: str, structure: Dict[str, Any]) -> str:
        """Classe le type de séance"""
        if 'vo2max' in query or 'max' in query:
            return 'VO2max'
        elif 'seuil' in query or 'threshold' in query:
            return 'Threshold'
        elif 'tempo' in query:
            return 'Tempo'
        elif structure['type'] == 'intervals':
            return 'Intervals'
        elif 'endurance' in query or 'aerobic' in query:
            return 'Endurance'
        else:
            return 'Mixed'
    
    def _has_warmup(self, query: str) -> bool:
        return bool(re.search(r'echauffement|chaude|chauffe|warm', query))
    
    def _has_cooldown(self, query: str) -> bool:
        return bool(re.search(r'retour|cool|calme', query))
    
    def _percent_to_zone(self, percent: int) -> str:
        """Convertit un pourcentage en zone"""
        if percent <= 55:
            return 'Zone 1'
        elif percent <= 75:
            return 'Zone 2'
        elif percent <= 90:
            return 'Zone 3'
        elif percent <= 105:
            return 'Zone 4'
        elif percent <= 120:
            return 'Zone 5'
        else:
            return 'Zone 6'

# Test d'extraction de paramètres
extractor = ParameterExtractor()

# Test avec notre requête corrigée
params = extractor.extract_workout_parameters(corrected_text)

print(f"\n🔍 PARAMÈTRES EXTRAITS:")
print(f"   Type de séance: {params['workout_type']}")
print(f"   Durées: {params['durations']}")
print(f"   Intensités: {list(params['intensities'].keys())}")
print(f"   Structure: {params['structure']['type']}")
print(f"   Échauffement: {'✅' if params['has_warmup'] else '❌'}")
print(f"   Retour au calme: {'✅' if params['has_cooldown'] else '❌'}")


In [ ]:
class ParametricWorkoutGenerator:
    """Générateur de séances paramétriques - Version simplifiée pour demo"""
    
    def __init__(self, extractor: ParameterExtractor):
        self.extractor = extractor
    
    def generate_workout(self, corrected_query: str) -> ParametricWorkout:
        """Génère une séance paramétrique à partir de la requête"""
        params = self.extractor.extract_workout_parameters(corrected_query)
        
        segments = []
        total_duration = 0
        
        # 1. Échauffement (par défaut ou spécifié)
        if params['has_warmup'] or not params['durations']['warmup']:
            warmup_duration = params['durations']['warmup'][0] if params['durations']['warmup'] else 10
        else:
            warmup_duration = 10
            
        if warmup_duration > 0:
            segments.append(WorkoutSegment(
                name="Échauffement",
                duration_minutes=warmup_duration,
                intensity_percent=60,
                zone="Zone 2",
                description="Échauffement progressif",
                segment_type="warmup"
            ))
            total_duration += warmup_duration
        
        # 2. Travail principal
        if params['structure']['type'] == 'intervals' and params['durations']['work']:
            # Séances par intervalles
            work_durations = params['durations']['work']
            recovery_durations = params['durations']['recovery']
            
            # Intensité de travail
            work_intensity = self._get_work_intensity(params['intensities'])
            
            for i, work_duration in enumerate(work_durations):
                # Segment de travail
                segments.append(WorkoutSegment(
                    name=f"Travail {i+1}",
                    duration_minutes=work_duration,
                    intensity_percent=work_intensity,
                    zone=self._intensity_to_zone(work_intensity),
                    description=f"Série {i+1} - {params['workout_type']}",
                    segment_type="work"
                ))
                total_duration += work_duration
                
                # Récupération (sauf après la dernière série)
                if i < len(work_durations) - 1 and i < len(recovery_durations):
                    recovery_duration = recovery_durations[i]
                    segments.append(WorkoutSegment(
                        name=f"Récupération {i+1}",
                        duration_minutes=recovery_duration,
                        intensity_percent=50,
                        zone="Zone 1",
                        description="Récupération active",
                        segment_type="recovery"
                    ))
                    total_duration += recovery_duration
        else:
            # Travail continu
            work_duration = max(20, sum(params['durations']['work']) if params['durations']['work'] else 30)
            work_intensity = self._get_work_intensity(params['intensities'])
            
            segments.append(WorkoutSegment(
                name="Travail Principal",
                duration_minutes=work_duration,
                intensity_percent=work_intensity,
                zone=self._intensity_to_zone(work_intensity),
                description=f"Travail continu - {params['workout_type']}",
                segment_type="work"
            ))
            total_duration += work_duration
        
        # 3. Retour au calme
        if params['has_cooldown'] or not params['durations']['cooldown']:
            cooldown_duration = params['durations']['cooldown'][0] if params['durations']['cooldown'] else 10
        else:
            cooldown_duration = 10
            
        if cooldown_duration > 0:
            segments.append(WorkoutSegment(
                name="Retour au Calme",
                duration_minutes=cooldown_duration,
                intensity_percent=50,
                zone="Zone 1",
                description="Retour au calme progressif",
                segment_type="cooldown"
            ))
            total_duration += cooldown_duration
        
        # Création de la séance complète
        workout_name = f"Séance {params['workout_type']} - {total_duration}min"
        
        return ParametricWorkout(
            name=workout_name,
            description=corrected_query,
            total_duration=total_duration,
            segments=segments,
            difficulty=self._calculate_difficulty(params, segments),
            workout_type=params['workout_type']
        )
    
    def _get_work_intensity(self, intensities: Dict[str, Any]) -> int:
        """Détermine l'intensité de travail principale"""
        if 'explicit_percent' in intensities:
            return intensities['explicit_percent']['target_power']
        elif intensities:
            # Prend la première intensité trouvée
            first_intensity = next(iter(intensities.values()))
            return first_intensity['target_power']
        else:
            return 85  # Défaut Zone 3-4
    
    def _intensity_to_zone(self, intensity_percent: int) -> str:
        """Convertit une intensité en zone"""
        if intensity_percent <= 55:
            return 'Zone 1'
        elif intensity_percent <= 75:
            return 'Zone 2'
        elif intensity_percent <= 90:
            return 'Zone 3'
        elif intensity_percent <= 105:
            return 'Zone 4'
        elif intensity_percent <= 120:
            return 'Zone 5'
        else:
            return 'Zone 6'
    
    def _calculate_difficulty(self, params: Dict[str, Any], segments: List[WorkoutSegment]) -> int:
        """Calcule la difficulté de la séance (1-5)"""
        max_intensity = max([s.intensity_percent for s in segments if s.segment_type == 'work'], default=50)
        
        if max_intensity <= 70:
            return 1  # Facile
        elif max_intensity <= 85:
            return 2  # Modéré
        elif max_intensity <= 95:
            return 3  # Difficile
        elif max_intensity <= 110:
            return 4  # Très difficile
        else:
            return 5  # Extrême

# Test de génération paramétrique
generator = ParametricWorkoutGenerator(extractor)
workout = generator.generate_workout(corrected_text)

print(f"\n🏗️ SÉANCE GÉNÉRÉE:")
print(f"   Nom: {workout.name}")
print(f"   Durée totale: {workout.total_duration} min")
print(f"   Difficulté: {workout.difficulty}/5")
print(f"   Type: {workout.workout_type}")
print(f"\n📋 SEGMENTS:")
for i, segment in enumerate(workout.segments):
    print(f"   {i+1}. {segment.name} - {segment.duration_minutes}min à {segment.intensity_percent}%FTP ({segment.zone})")


In [ ]:
# Génération de fichier .zwo simplifié
import os
import time

class SimpleZwoGenerator:
    """Générateur .zwo simplifié pour demo"""
    
    def generate_zwo(self, workout: ParametricWorkout, output_dir: str = "./generated_workouts") -> str:
        """Génère un fichier .zwo à partir d'une séance paramétrique"""
        os.makedirs(output_dir, exist_ok=True)
        
        # Nom de fichier unique
        timestamp = int(time.time())
        filename = f"demo_workout_{timestamp}.zwo"
        filepath = os.path.join(output_dir, filename)
        
        # Génération du contenu XML
        xml_content = self._generate_xml_content(workout)
        
        # Écriture du fichier
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(xml_content)
        
        return filepath
    
    def _generate_xml_content(self, workout: ParametricWorkout) -> str:
        """Génère le contenu XML du fichier .zwo"""
        
        # Header XML
        xml = '<?xml version="1.0" encoding="UTF-8"?>\n'
        xml += '<workout_file>\n'
        xml += f'    <author>Vekta AI Demo</author>\n'
        xml += f'    <name>{workout.name}</name>\n'
        xml += f'    <description>{workout.description}</description>\n'
        xml += '    <sportType>bike</sportType>\n'
        xml += '    <tags>\n'
        xml += '        <tag name="Vekta"/>\n'
        xml += '        <tag name="Parametric"/>\n'
        xml += '        <tag name="Demo"/>\n'
        xml += '    </tags>\n'
        xml += '    <workout>\n'
        
        # Segments
        for segment in workout.segments:
            duration_seconds = segment.duration_minutes * 60
            power_decimal = segment.intensity_percent / 100.0
            
            if segment.segment_type == 'warmup':
                xml += f'        <Warmup Duration="{duration_seconds}" PowerLow="0.50" PowerHigh="{power_decimal:.2f}"/>\n'
            elif segment.segment_type == 'cooldown':
                xml += f'        <Cooldown Duration="{duration_seconds}" PowerHigh="{power_decimal:.2f}" PowerLow="0.50"/>\n'
            elif segment.segment_type == 'work':
                xml += f'        <SteadyState Duration="{duration_seconds}" Power="{power_decimal:.2f}"/>\n'
            elif segment.segment_type == 'recovery':
                xml += f'        <SteadyState Duration="{duration_seconds}" Power="{power_decimal:.2f}"/>\n'
        
        xml += '    </workout>\n'
        xml += '</workout_file>\n'
        
        return xml

# Test de génération .zwo
zwo_generator = SimpleZwoGenerator()
zwo_file = zwo_generator.generate_zwo(workout)

print(f"\n💾 FICHIER .ZWO GÉNÉRÉ:")
print(f"   Fichier: {zwo_file}")

# Affichage du contenu généré
with open(zwo_file, 'r', encoding='utf-8') as f:
    content = f.read()

print(f"\n📄 CONTENU DU FICHIER .ZWO (extrait):")
lines = content.split('\n')
for i, line in enumerate(lines[:15]):  # Première partie seulement
    print(line)
if len(lines) > 15:
    print("    ...")


In [ ]:
class HybridPipeline:
    """Pipeline hybride : RAG + Génération Paramétrique"""
    
    def __init__(self):
        self.spell_checker = CyclingSpellChecker()
        self.extractor = ParameterExtractor()
        self.generator = ParametricWorkoutGenerator(self.extractor)
        self.zwo_generator = SimpleZwoGenerator()
    
    def process_query(self, query: str, use_parametric: bool = True) -> Dict[str, Any]:
        """Traite une requête avec choix RAG ou génération paramétrique"""
        
        # 1. Correction orthographique
        corrected_text, corrections = self.spell_checker.correct_text(query)
        
        # 2. Choix du mode de traitement
        if use_parametric:
            # Mode génération paramétrique (pour coachs experts)
            workout = self.generator.generate_workout(corrected_text)
            
            # Génération du fichier .zwo
            zwo_file = self.zwo_generator.generate_zwo(workout)
            
            return {
                'mode': 'parametric',
                'success': True,
                'confidence': 0.95,  # Haute confiance en mode paramétrique
                'workout': {
                    'name': workout.name,
                    'description': workout.description,
                    'duration_minutes': workout.total_duration,
                    'difficulty': workout.difficulty,
                    'type': workout.workout_type,
                    'segments': [
                        {
                            'name': s.name,
                            'duration': s.duration_minutes,
                            'intensity': s.intensity_percent,
                            'zone': s.zone,
                            'type': s.segment_type,
                            'description': s.description
                        }
                        for s in workout.segments
                    ]
                },
                'zwo_file': zwo_file,
                'corrections': corrections,
                'message': f"Séance générée paramétriquement - {workout.total_duration}min, difficulté {workout.difficulty}/5"
            }
        else:
            # Mode RAG classique (simulation pour demo)
            return {
                'mode': 'rag',
                'success': True,
                'confidence': 0.75,
                'workout': None,
                'message': "Mode RAG non implémenté dans cette demo",
                'corrections': corrections
            }

# Test du pipeline hybride
pipeline = HybridPipeline()

# Test en mode paramétrique
result = pipeline.process_query(test_query, use_parametric=True)

print(f"\n🔄 RÉSULTAT DU PIPELINE HYBRIDE:")
print(f"   Mode: {result['mode']}")
print(f"   Succès: {result['success']}")
print(f"   Confiance: {result['confidence']:.0%}")
print(f"   Message: {result['message']}")
print(f"   Fichier .zwo: {result['zwo_file']}")

if result['workout']:
    print(f"\n🏃 DÉTAILS DE LA SÉANCE:")
    print(f"   Nom: {result['workout']['name']}")
    print(f"   Durée: {result['workout']['duration_minutes']}min")
    print(f"   Difficulté: {result['workout']['difficulty']}/5")
    print(f"   Segments: {len(result['workout']['segments'])}")

print(f"\n✅ DEMO GÉNÉRATION PARAMÉTRIQUE TERMINÉE")
print(f"   • Correction orthographique ✅")
print(f"   • Extraction paramètres ✅")
print(f"   • Génération séance ✅")
print(f"   • Export .zwo ✅")
print(f"   • Pipeline hybride ✅")
